In [1]:
import arcpy
import os

In [2]:
arcpy.env.workspace
arcpy.env.overwriteOutput = True 

## Add the Sun Cloud Boundary/Counties ('SunCloud')

In [3]:
inFeatures = 'https://services3.arcgis.com/gjVvdAtTsjMYfRZ8/arcgis/rest/services/Sun_Cloud_Counties/FeatureServer/6'
outLocation = r'C:\Users\YousefDana\Documents\git\sun_cloud\layers\Junctions'
outFeatureClass = "SunCloud"

arcpy.FeatureClassToFeatureClass_conversion(inFeatures, outLocation, 
                                            outFeatureClass)

<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Junctions\\SunCloud.shp'>

## Add Routes ('sun_cloud_routes')

In [6]:
inFeatures = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Routes/FeatureServer/6'
outLocation = r'C:\Users\YousefDana\Documents\git\sun_cloud\layers\Junctions'
outFeatureClass = 'sun_cloud_routes'

arcpy.FeatureClassToFeatureClass_conversion(inFeatures, outLocation, 
                                            outFeatureClass)

<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Junctions\\sun_cloud_routes.shp'>

## Add Junctions ('Junctions')

In [16]:
inFeatures = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Arizona_Highway_Performance_Monitoring_System_2020_Data_/FeatureServer/0'
outLocation = r'C:\Users\YousefDana\Documents\git\sun_cloud\layers\Junctions'
outFeatureClass = "Junctions_orig"

arcpy.FeatureClassToFeatureClass_conversion(inFeatures, outLocation, 
                                            outFeatureClass)

<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Junctions\\Junctions_orig.shp'>

## Add Segments Layer ('Major_Intersections')

In [29]:
# Find major junctions depending on intersection of major routes
arcpy.conversion.ExportFeatures("sun_cloud_routes", "sun_cloud_routes_backup")
route_raw = 'sun_cloud_routes'
sql = """functional >= 1 And functional <= 7 Or functional >= 11 And functional <= 17"""
selected = arcpy.SelectLayerByAttribute_management(route_raw, "NEW_SELECTION", sql, "INVERT")
count = int(arcpy.GetCount_management(selected)[0])
print (count)

if count > 0:
    arcpy.DeleteFeatures_management(selected)
    
# Find the real intersections. 
# Reference: unsplit the original route to find the real intersections 
# (https://support.esri.com/en/technical-article/000025044)
arcpy.management.UnsplitLine(route_raw, 
                             r"C:\Users\YousefDana\Documents\git\sun_cloud\layers\Safety\Safety.gdb\unsplit_route", 
                             None, None, '')
# Run the Intersect tool to create points at interaction
arcpy.analysis.Intersect("unsplit_route", "major_intersection", "ALL", None, "POINT")
arcpy.management.MultipartToSinglepart('major_intersection', 'Major_Intersections')


0


<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Safety\\Safety.gdb\\major_intersec_single'>

## Clip Junctions by Sun Cloud Boundary ('Junctions_SunCloud_Clip')

In [16]:
arcpy.analysis.Clip("Junctions", 
                    "SunCloud", 
                    r"C:\Users\YousefDana\Documents\git\sun_cloud\layers\Junctions\Junctions.gdb\Junctions_SunCloud_Clip", 
                    None)


<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Junctions\\Junctions.gdb\\Junctions_SunCloud_Clip'>

In [1]:
## Filter Junctions

In [11]:
# Select Layer by Attribute and Narrow Down to Desired Junctions
arcpy.management.SelectLayerByAttribute("Junctions_SunCloud_Clip", 
                                        "NEW_SELECTION", 
                                        "JunctionGe IN (0, 1, 2, 3, 4, 5) And JunctionsS = 'J' And (TrafficCon IN (4, 5, 6, 7) Or (TrafficCon IN (92, 94) And Signalizat < 5))", 
                                        None)

# Export the Selected Features to a New Feature Class ('Junctions_SunCloud_Select')
arcpy.conversion.FeatureClassToFeatureClass("Junctions_SunCloud_Clip", 
                                            r"C:\Users\YousefDana\Documents\git\sun_cloud\layers\Junctions\Junctions.gdb", 
                                            "Junctions_SunCloud_Select", 
                                            '', 
                                            'FromDate "FromDate" true true false 8 Date 0 0,First,#,Junctions_SunCloud,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,Junctions_SunCloud,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,Junctions_SunCloud,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,Junctions_SunCloud,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,Junctions_SunCloud,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,Junctions_SunCloud,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,Junctions_SunCloud,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,Junctions_SunCloud,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,Junctions_SunCloud,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,Junctions_SunCloud,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,Junctions_SunCloud,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,Junctions_SunCloud,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,Junctions_SunCloud,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,Junctions_SunCloud,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,Junctions_SunCloud,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,Junctions_SunCloud,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,Junctions_SunCloud,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,Junctions_SunCloud,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,Junctions_SunCloud,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,Junctions_SunCloud,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,Junctions_SunCloud,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,Junctions_SunCloud,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,Junctions_SunCloud,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,Junctions_SunCloud,InServiceD,-1,-1', 
                                            '')

<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Junctions\\Junctions.gdb\\Junctions_SunCloud_Select'>

## Intersect Junctions_SunCloud_Select with Routes ('Sun_Cloud_Junctions')

In [12]:
arcpy.analysis.PairwiseIntersect("Junctions_SunCloud_Select;sun_cloud_routes", 
                                 r"C:\Users\YousefDana\Documents\git\sun_cloud\layers\Junctions\Junctions.gdb\Sun_Cloud_Junctions", 
                                 "ALL", 
                                 None, 
                                 "POINT")

<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Junctions\\Junctions.gdb\\Sun_Cloud_Junctions'>

## Delete Identical Features by shape

In [13]:
arcpy.management.DeleteIdentical("Sun_Cloud_Junctions", "Shape", None, 0)

<Result 'Sun_Cloud_Junctions'>

## Select by Location

In [14]:
arcpy.management.SelectLayerByLocation("Sun_Cloud_Junctions", 
                                       "INTERSECT", 
                                       "major_intersections", 
                                       "50 Feet", 
                                       "NEW_SELECTION", 
                                       "NOT_INVERT")

<Result 'Sun_Cloud_Junctions'>

## Export Selection to New Feature Class (Sun_Cloud_Major_Junctions)

In [15]:
arcpy.conversion.FeatureClassToFeatureClass("Sun_Cloud_Junctions", 
                                            r"C:\Users\YousefDana\Documents\git\sun_cloud\layers\Junctions\Junctions.gdb", 
                                            "Sun_Cloud_Major_Junctions", 
                                            '', 
                                            'FID_Junctions_SunCloud_Select "FID_Junctions_SunCloud_Select" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,FID_Junctions_SunCloud_Select,-1,-1;FromDate "FromDate" true true false 8 Date 0 0,First,#,Sun_Cloud_Junctions,FromDate,-1,-1;ToDate "ToDate" true true false 8 Date 0 0,First,#,Sun_Cloud_Junctions,ToDate,-1,-1;RouteID "RouteID" true true false 32 Text 0 0,First,#,Sun_Cloud_Junctions,RouteID,0,32;Measure "Measure" true true false 8 Double 0 0,First,#,Sun_Cloud_Junctions,Measure,-1,-1;MasterJunc "MasterJunc" true true false 38 Text 0 0,First,#,Sun_Cloud_Junctions,MasterJunc,0,38;NumberOfLe "NumberOfLe" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,NumberOfLe,-1,-1;JunctionGe "JunctionGe" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,JunctionGe,-1,-1;JunctionsS "JunctionsS" true true false 2 Text 0 0,First,#,Sun_Cloud_Junctions,JunctionsS,0,2;TrafficCon "TrafficCon" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,TrafficCon,-1,-1;TurnCode "TurnCode" true true false 10 Text 0 0,First,#,Sun_Cloud_Junctions,TurnCode,0,10;Detection "Detection" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,Detection,-1,-1;Signalizat "Signalizat" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,Signalizat,-1,-1;RailCrossi "RailCrossi" true true false 7 Text 0 0,First,#,Sun_Cloud_Junctions,RailCrossi,0,7;SchoolZone "SchoolZone" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,SchoolZone,-1,-1;BicycleFac "BicycleFac" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,BicycleFac,-1,-1;Crosswalk "Crosswalk" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,Crosswalk,-1,-1;PctGreenTi "PctGreenTi" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,PctGreenTi,-1,-1;PctGreen_1 "PctGreen_1" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,PctGreen_1,-1,-1;SourceID "SourceID" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,SourceID,-1,-1;JunctionMa "JunctionMa" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,JunctionMa,-1,-1;JunctionTy "JunctionTy" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,JunctionTy,-1,-1;OffsetInte "OffsetInte" true true false 1 Text 0 0,First,#,Sun_Cloud_Junctions,OffsetInte,0,1;OffsetIn_1 "OffsetIn_1" true true false 8 Double 0 0,First,#,Sun_Cloud_Junctions,OffsetIn_1,-1,-1;InServiceD "InServiceD" true true false 8 Date 0 0,First,#,Sun_Cloud_Junctions,InServiceD,-1,-1;FID_sun_cloud_routes "FID_sun_cloud_routes" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,FID_sun_cloud_routes,-1,-1;route_id "route_id" true true false 32 Text 0 0,First,#,Sun_Cloud_Junctions,route_id,0,32;from_measu "from_measu" true true false 8 Double 0 0,First,#,Sun_Cloud_Junctions,from_measu,-1,-1;to_measure "to_measure" true true false 8 Double 0 0,First,#,Sun_Cloud_Junctions,to_measure,-1,-1;functional "functional" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,functional,-1,-1;facility_t "facility_t" true true false 4 Long 0 0,First,#,Sun_Cloud_Junctions,facility_t,-1,-1;Shape__Len "Shape__Len" true true false 8 Double 0 0,First,#,Sun_Cloud_Junctions,Shape__Len,-1,-1', 
                                            '')

<Result 'C:\\Users\\YousefDana\\Documents\\git\\sun_cloud\\layers\\Junctions\\Junctions.gdb\\Sun_Cloud_Major_Junctions'>

## Rename Feature Layer Fields

In [32]:
layer = 'Sun_Cloud_Major_Junctions'
system_fields = ['OBJECTID', 'Shape']
current_fields = [f.name for f in arcpy.ListFields(layer) if not f.name in system_fields]
target_fields = [('FromDate', 'from_date', 'From Date'),
                 ('ToDate', 'to_date', 'To Date'),
                 ('RouteID', 'route_identification', 'Route Identification'),
                 ('Measure', 'measure', 'Measure'),
                 ('MasterJunc', 'master_junction_id', 'Master Junction ID'),
                 ('NumberOfLe', 'number_of_legs', 'Number of Legs'),
                 ('JunctionGe', 'junction_geometry', 'Junction Geometry'),
                 ('JunctionsS', 'junctions_symbol', 'Junctions Symbol'),
                 ('TrafficCon', 'traffic_control', 'Traffic Control'),
                 ('TurnCode', 'turn_code', 'Turn Code'),
                 ('Detection', 'detection', 'Detection'),
                 ('Signalizat', 'signalization', 'Signalization'),
                 ('RailCrossi', 'rail_crossing', 'Rail Crossing'),
                 ('SchoolZone', 'school_zone', 'School Zone'),
                 ('BicycleFac', 'bicycle_facility', 'Bicycle Facility'),
                 ('Crosswalk', 'crosswalk', 'Crosswalk'),
                 ('PctGreenTi', 'percent_green_time_on_road', 'Percent Green Time On Road'),
                 ('PctGreen_1', 'percent_green_time_source', 'Percent Green Time Source'),
                 ('JunctionMa', 'junction_maint_by', 'Junction Maintained By'),
                 ('JunctionTy', 'junction_type', 'Junction Type'),
                 ('OffsetInte', 'offset_intersection', 'Offset Intersection'),
                 ('OffsetIn_1', 'offset_intersection_distance', 'Offset Intersection Distance'),
                 ('InServiceD', 'in_service_date', 'In Service Date')]

delete_fields = [field for field in current_fields if not field in [a for a, b, c in target_fields]]
# delete unused fields
arcpy.management.DeleteField(layer, delete_fields)
for current_name, new_name, alias in target_fields:
    arcpy.management.AlterField(layer, current_name, new_name, alias)